# 데이터 전처리 2

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import mglearn

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 Feature Engineering
- 구간분할, 이산화
- 상호작용과 다항식 특성
- 특성 자동 선택

### 1.1 구간 분할, 이산화 - Binning

#### 1.1.1 wave 데이터셋에 적용한 선형 회귀와 결정 트리의 비교

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# 데이터 생성
X, y = mglearn.datasets.make_wave(n_samples=120)
line = np.linspace(-3, 3, 1000, endpoint=False).reshape(-1, 1)

# 선형 회귀
reg = LinearRegression().fit(X, y)
plt.plot(line, reg.predict(line), '--', label='선형 회귀')

# 결정 트리
reg = DecisionTreeRegressor(min_samples_leaf=3).fit(X, y)
plt.plot(line, reg.predict(line), label='결정 트리')

# 원본 데이터 (X, y)
plt.plot(X[:, 0], y, 'o', c='k')

plt.title('wave 데이터셋에 적용한 선형 회귀와 결정 트리의 비교')
plt.xlabel('입력 특성')
plt.ylabel('회귀 출력')
plt.legend(loc='best')
plt.show()

#### 1.1.2 구간 분할( binning) - 밀집 행렬

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
# <참고>
# 희소 행렬로 생성시 밀집 행렬로 변환해서 사용
# X_binned.toarray() 함수 이용

# 희소 행렬로 생성
kb = KBinsDiscretizer(n_bins=10, strategy='uniform')
kb.fit(X)
X_binned = kb.transform(X)

# 밀집 행렬로 변환
X_binned.toarray()

In [ ]:
# 구간 분할 모델 생성
kb = KBinsDiscretizer(n_bins=10, strategy='uniform', encode='onehot-dense')

In [ ]:
# 구간 분할 모델 학습 - 경계값 생성
kb.fit(X)

In [ ]:
# 구간 분할 모델 데이터 변환
X_binned = kb.transform(X)

In [ ]:
# 구간 경계값
kb.bin_edges_

In [ ]:
# 원본 데이터
X[:10]

In [ ]:
# 구간 분할 적용된 데이터
X_binned[:10]

#### 1.1.3 구간 분할된 특성에 적용한 선형 회귀와 결정 트리 회귀의 비교
- 각 구간에 기울기가 0인 수평한 회귀선 생성

In [ ]:
# 구간 분할 적용(line)
line_binned = kb.transform(line)

# 선형 회귀
reg = LinearRegression().fit(X_binned, y)
plt.plot(line, reg.predict(line_binned), '--', label='구간 선형 회귀')

# 결정 트리
reg = DecisionTreeRegressor(min_samples_split=3).fit(X_binned, y)
plt.plot(line, reg.predict(line_binned), label='구간 결정 트리')

# 원본 데이터 (X, y)
plt.plot(X[:, 0], y, 'o', c='k')

# 구간 분할 표시
plt.vlines(kb.bin_edges_[0], -3, 3, linewidth=1, alpha=0.2)

plt.title('구간 분할된 특성에 적용한 선형 회귀와 결정 트리 회귀의 비교')
plt.xlabel('입력 특성')
plt.ylabel('회귀 출력')
plt.legend(loc='best')
plt.show()

### 1.2 상호작용과 다항식 특성 - Interaction & Polynomial

#### 1.2.1 원본 특성 추가
- X + X_binned
- 구간 분할된 특성과 하나의 기울기를 사용한 선형 회귀

In [ ]:
# X값 추가
X_combined = np.hstack([X, X_binned])
X_combined.shape

In [ ]:
# 구간 분할 및 원본 특성 추가 적용(line)
line_combined = np.hstack([line, line_binned])

# 선형 회귀
reg = LinearRegression().fit(X_combined, y)
plt.plot(line, reg.predict(line_combined), label='원본 특성을 추가한 선형 회귀')

# 원본 데이터 (X, y)
plt.plot(X[:, 0], y, 'o', c='k')

# 구간 분할 표시
plt.vlines(kb.bin_edges_[0], -3, 3, linewidth=1, alpha=0.2)

plt.title('구간 분할된 특성과 하나의 기울기를 사용한 선형 회귀')
plt.xlabel('입력 특성')
plt.ylabel('회귀 출력')
plt.legend(loc='best')
plt.show()

#### 1.2.2 원본 특성을 곱한 값 추가
- X_binned + (X * X_binned)
- 구간별 기울기가 다른 선형 회귀

In [ ]:
# X값 추가
X_product = np.hstack([X_binned, X * X_binned])
X_product.shape

In [ ]:
# 구간 분할 및 원본 특성 추가 적용(line)
line_product = np.hstack([line_binned, line * line_binned])

# 선형 회귀
reg = LinearRegression().fit(X_product, y)
plt.plot(line, reg.predict(line_product), label='원본 특성을 곱한 값을 추가한 선형 회귀')

# 원본 데이터 (X, y)
plt.plot(X[:, 0], y, 'o', c='k')

# 구간 분할 표시
plt.vlines(kb.bin_edges_[0], -3, 3, linewidth=1, alpha=.2)

plt.title('구간 분할된 특성과 서로 기울기를 사용한 선형 회귀')
plt.xlabel('입력 특성')
plt.ylabel('회귀 출력')
plt.legend(loc='best')
plt.show()

#### 1.2.3 다항식 특성
- PolynomialFeatures

In [ ]:
# 다항식 특성 추가 모델 생성
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=10, include_bias=False) # 절편 추가: include_bias=True

In [ ]:
# 다항식 특성 추가 모델 학습
poly.fit(X)

In [ ]:
# 다항식 특성 추가 모델 데이터 변환
X_poly = poly.transform(X)
X_poly.shape

In [ ]:
# 원본 데이터
X[:5]

In [ ]:
# 다항식 특성 추가 적용된 데이터
X_poly[:5]

In [ ]:
# 컬럼 이름
print(poly.get_feature_names_out())

In [ ]:
# 판다스(DataFrame)로 표현
pd.DataFrame(X_poly, columns=poly.get_feature_names_out())

In [ ]:
# 다항식 특성 추가 적용(line)
line_poly = poly.transform(line)

# 선형 회귀
reg = LinearRegression().fit(X_poly, y)
plt.plot(line, reg.predict(line_poly), label='다항 선형 회귀')

# 원본 데이터 (X, y)
plt.plot(X[:, 0], y, 'o', c='k')

plt.title('10차 다항식을 이용한 선형 회귀')
plt.xlabel('입력 특성')
plt.ylabel('회귀 출력')
plt.legend(loc='best')
plt.show()

#### 1.2.4 원본 데이터에 SVM 알고리즘 적용 - 다항 회귀와 비교

In [ ]:
from sklearn.svm import SVR

# Support Vector Machine
for gamma in [1, 10]:
    svr = SVR(gamma=gamma).fit(X, y)
    plt.plot(line, svr.predict(line), label=f'SVR gamma={gamma}')

# 원본 데이터 (X, y)
plt.plot(X[:, 0], y, 'o', c='k')

plt.title('RBF 커널 SVM의 gamma 매개변수 변화에 따른 비교')
plt.xlabel('입력 특성')
plt.ylabel('회귀 출력')
plt.legend(loc='best')
plt.show()

#### 1.2.5 다항식 특성 추가 적용 - 지도학습

In [ ]:
# 데이터 로딩
df_boston = pd.read_csv('data/boston.csv')
X = df_boston.drop('target', axis=1).values
y = df_boston['target'].values

In [ ]:
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
# 데이터 스케일 조정
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 다항식 특성 추가
poly = PolynomialFeatures(degree=2).fit(X_train_scaled)
X_train_poly = poly.transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

print("X_train.shape:", X_train.shape)
print("X_train_poly.shape:", X_train_poly.shape)

In [ ]:
# 컬럼 이름
print(poly.get_feature_names_out())

In [ ]:
#
# 다항식 특성 추가 성능 비교 - 회귀 계열 알고리즘
#

from sklearn.linear_model import Ridge

ridge = Ridge().fit(X_train_scaled, y_train)
print('다항식 특성 추가 미적용: {:.3f}'.format(ridge.score(X_test_scaled, y_test)))

ridge = Ridge().fit(X_train_poly, y_train)
print('다항식 특성 추가 적용:   {:.3f}'.format(ridge.score(X_test_poly, y_test)))

In [ ]:
#
# 다항식 특성 추가 성능 비교 - 트리 계열 알고리즘
#

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=123).fit(X_train_scaled, y_train)
print('다항식 특성 추가 미적용: {:.3f}'.format(rf.score(X_test_scaled, y_test)))

rf = RandomForestRegressor(n_estimators=100, random_state=123).fit(X_train_poly, y_train)
print('다항식 특성 추가 적용:   {:.3f}'.format(rf.score(X_test_poly, y_test)))

### 1.3 특성 자동 선택
- 일변량 통계 (Univariate statistics)
- 모델 기반 선택 (Model-based selection)
- 반복적 선택 (Iterative selection)

#### 1.3.1 일변량 통계

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 로딩
cancer = load_breast_cancer()

In [ ]:
# 노이즈 생성: 50개
rng = np.random.RandomState(123)
noise = rng.normal(size=(len(cancer.data), 50))

# 데이터에 노이즈 특성 추가: 원본특성(30 + 노이즈(50)
X_w_noise = np.hstack([cancer.data, noise])

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_w_noise, cancer.target, random_state=123, test_size=0.5)

In [ ]:
# 특성 선택 모델 생성: 일변량 통계 (분산 분석 사용)
select = SelectPercentile(score_func=f_classif, percentile=50)

In [ ]:
# 특성 선택 모델 학습
select.fit(X_train, y_train)

In [ ]:
# 특성 선택 데이터 변환: 적절한 특성(컬럼) 선택
X_train_selected = select.transform(X_train)

print('X_train.shape:', X_train.shape)
print('X_train_selected.shape:', X_train_selected.shape)

In [ ]:
# 특성 선택 여부 표시
mask = select.get_support()
mask

In [ ]:
# True는 검은색, False는 흰색으로 표시
plt.matshow(mask.reshape(1, -1), cmap='gray_r')

plt.title('SelectPercentile이 선택한 특성')
plt.xlabel('특성 번호')
plt.yticks([0])
plt.show()

In [ ]:
#
# 일변량 통계 특성 선택 성능 비교 - Logistic Regression
#

from sklearn.linear_model import LogisticRegression

# 테스트 데이터 변환: 특성 선택
X_test_selected = select.transform(X_test)

# LogisticRegression
lr = LogisticRegression(max_iter=5000)

lr.fit(X_train, y_train)
print('전체 특성 사용: {:.3f}'.format(lr.score(X_test, y_test)))

lr.fit(X_train_selected, y_train)
print('일변량 통계 특성 선택 적용(일부 특성 사용): {:.3f}'.format(lr.score(X_test_selected, y_test)))

#### 1.3.2 모델 기반 특성 선택

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# 특성 선택 모델 생성: 모델 기반 특성 선택
select = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=123), threshold='median')

In [ ]:
# 특성 선택 모델 학습
select.fit(X_train, y_train)

In [ ]:
# 특성 선택 데이터 변환: 적절한 특성(컬럼) 선택
X_train_l1 = select.transform(X_train)

print('X_train.shape:   ', X_train.shape)
print('X_train_l1.shape:', X_train_l1.shape)

In [ ]:
# 특성 선택 여부 표시
mask = select.get_support()
mask

In [ ]:
# True는 검은색, False는 흰색으로 표시
plt.matshow(mask.reshape(1, -1), cmap='gray_r')

plt.title('랜덤 포레스트 분류 모델을 사용한 SelectFromModel이 선택한 특성')
plt.xlabel('특성 번호')
plt.yticks([0])
plt.show()

In [ ]:
#
# 모델 기반 특성 선택 성능 비교 - Logistic Regression
#

from sklearn.linear_model import LogisticRegression

# 테스트 데이터 변환: 특성 선택
X_test_l1 = select.transform(X_test)

# LogisticRegression
lr = LogisticRegression(max_iter=5000)

lr.fit(X_train_l1, y_train)
print('모델 기반 특성 선택 적용(일부 특성 사용): {:.3f}'.format(lr.score(X_test_l1, y_test)))

#### 1.3.3 반복적 특성 선택
- 재귀적 특성 제거 (RFE - Recursive Feature Elimination)

In [ ]:
# 특성 선택 모델 생성: 반복적 특성 기반
from sklearn.feature_selection import RFE
select = RFE(RandomForestClassifier(n_estimators=100, random_state=123), n_features_to_select=40)

In [ ]:
%%time
# 특성 선택 모델 학습
select.fit(X_train, y_train)

In [ ]:
# 특성 선택 데이터 변환: 적절한 특성(컬럼) 선택
X_train_rfe = select.transform(X_train)

print('X_train.shape:    ', X_train.shape)
print('X_train_rfe.shape:', X_train_rfe.shape)

In [ ]:
# 특성 선택 여부 표시
mask = select.get_support()
mask

In [ ]:
# True는 검은색, False는 흰색으로 표시
plt.matshow(mask.reshape(1, -1), cmap='gray_r')

plt.title('랜덤 포레스트 분류 모델을 사용한 RFE가 선택한 특성')
plt.xlabel('특성 번호')
plt.yticks([0])
plt.show()

In [ ]:
#
# 반복적 특성 선택 기반 성능 비교 - Logistic Regression
#

from sklearn.linear_model import LogisticRegression

# 테스트 데이터 변환: 특성 선택
X_test_rfe = select.transform(X_test)

# LogisticRegression
lr = LogisticRegression(max_iter=5000)

lr.fit(X_train_rfe, y_train)
print('반복적 특성 선택 적용(일부 특성 사용): {:.3f}'.format(lr.score(X_test_rfe, y_test)))

In [ ]:
print('랜덤 포레스트(전체 특성 사용): {:.3f}'.format(select.score(X_test, y_test)))

# 2 Missing Value - 결측치

#### NaN 처리 메서드

인자 | 설명
:---|:---
dropna | 누락된 데이터가 있는 축(로우, 칼럼)을 제외시킨다. 어느 정도의 누락 데이터까지 용인할 것인지 지정할 수 있다.
fillna | 누락된 데이터를 대신할 값을 채우거나 'ffill' 또는 'bfill' 같은 보간 메서드를 적용한다.
isnull | 누락되거나 NA인 값을 알려주는 불리언 값이 저장된, 같은 형의 객체를 반환한다.
notnull | isnull과 반대되는 메서드다.

### 2.1 결측치 삭제

In [ ]:
df = pd.DataFrame([[1, 6.5, 3, np.nan],
                   [1, np.nan, np.nan, np.nan],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 6.5, 3, np.nan]])
df

In [ ]:
# NaN이 하나라도 있으면 drop
df.dropna()

In [ ]:
# 모든 값이 NaN인 로우만 삭제
df.dropna(how='all')

In [ ]:
# 행방향 처리
df.dropna(axis=0, how='all')

In [ ]:
# 열방향 처리
df.dropna(axis=1, how='all')

In [ ]:
df.dropna(thresh=3)

### 2.2 결측치 채우기

#### fillna 함수 인자

인자 | 설명
:---|:---
value | 비어있는 값을 채울 스칼라 값이나 사전 형식의 객체
method | 보간 방식. 기본적으로 'ffill'을 사용한다.
axis | 값을 채워 넣을 축. 기본 값은 0
inplace | 복사본을 생성하지 않고 호출한 객체를 변경한다. 기본값은 False
limit | 값을 앞 혹은 뒤에서 몇 개까지 채울지를 지정한다.

In [ ]:
#df.fillna(0, inplace=True) # 원본 수정
df.fillna(0)

In [ ]:
# 컬럼별로 값 지정
df.fillna({1: 0.5, 3: -1})

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.loc[2:, 1] = np.nan
df.loc[4:, 2] = np.nan
df

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='ffill', limit=2)

In [ ]:
sr = pd.Series([1., np.nan, 3.5, np.nan, 7])
sr

In [ ]:
sr.fillna(sr.mean())

# 3 Outlier - 이상치

### 3.1 Define function - vp_drop_outlier()

In [ ]:
def vp_drop_outlier(df, col, weight=1.5):
    sr = df[col]
    
    q25 = np.percentile(sr.values, 25)
    q75 = np.percentile(sr.values, 75)
    
    iqr   = q75 - q25
    iqr_w = iqr * weight
    
    val_l = q25 - iqr_w
    val_h = q75 + iqr_w
    
    outlier_index = sr[(sr < val_l) | (sr > val_h)].index
    
    df_res = df.drop(outlier_index).copy()
    
    return df_res

#### 3.1.1 Create DataFrame

In [ ]:
df = pd.DataFrame({'col1': range(100,110),
                   'col2': range(100,110),
                   'col3': range(100,110)})
df.loc[0,'col1'] = 95
df.loc[1,'col1'] = 115

df.loc[2,'col2'] = 92
df.loc[3,'col2'] = 117

df.loc[4,'col3'] = 90
df.loc[5,'col3'] = 120

In [ ]:
df

#### 3.1.2 Box Plot

In [ ]:
sns.boxplot(data=df)
plt.show()

#### 3.1.3 drop outlier - col1

In [ ]:
vp_drop_outlier(df,'col1')

#### 3.1.4 drop outlier - col2

In [ ]:
vp_drop_outlier(df,'col2')

#### 3.1.5 drop outlier - col3

In [ ]:
vp_drop_outlier(df,'col3')

# 4 Data Sampling
- Under Sampling
- Over Sampling - SMOTE

### 4.1 데이터 로딩

In [ ]:
df = pd.read_csv('./data/creditcard.csv')
df.drop('Time', axis=1, inplace=True)
df

In [ ]:
# 0: 정상 거래, 1: 사기 거래
df['Class'].value_counts()

### 4.2 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=123)

In [ ]:
# 0: 정상 거래, 1: 사기 거래
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
pd.Series(y_train).value_counts()

### 4.3 Over Sampling 적용 - SMOTE

In [ ]:
#!pip install imbalanced-learn

#### 4.3.1 SMOTE 모델 생성

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=123)

#### 4.3.2 데이터 생성

In [ ]:
X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

#### 4.3.3 데이터 확인

In [ ]:
# 0: 정상 거래, 1: 사기 거래
print(f'X_train_over: {X_train_over.shape}, y_train_over: {y_train_over.shape}')
pd.Series(y_train_over).value_counts()

### 4.4 Over Sampling 적용 후 분류 모델 평가

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

# 모델 생성
rf = RandomForestClassifier(n_estimators=5, random_state=123)

# 모델 학습
rf.fit(X_train_over, y_train_over)

# 모델 평가
rf.score(X_test, y_test)

---

In [ ]:
# End of file